In [12]:
import aerosandbox as asb
import numpy as np
import pandas as pd

In [13]:
df = pd.read_excel(r'C:\Users\Rafael\Downloads\airfoil_data_to_diff.xlsx')

In [14]:
airfoils_names = df['AIRFOIL_NAME'].tolist()

In [15]:
def cosine_spacing(n_points):
    """
    Generates a linspace from 0 to 1 using the cosine spacing distribution.
    
    Args:
    - n_points: Number of points in the linspace.
    
    Returns:
    - Array of n_points spaced according to the cosine distribution from 0 to 1.
    """
    
    i = np.arange(n_points)
    x_i = 0.5 * (1 - np.cos(np.pi * i / (n_points - 1)))
    
    return x_i

# Example usage:
n = 150
x = cosine_spacing(n)

In [16]:
from scipy.interpolate import interp1d

x_values = np.linspace(0, 1, 150)
# Create dictionaries to store the interpolation functions
upper_interp_funcs = {}
lower_interp_funcs = {}

# Loop through each airfoil name
for airfoil in airfoils_names:
    # Filter dataframe for the current airfoil
    airfoil_df = df[df['AIRFOIL_NAME'] == airfoil]
    
    # Extract y values for the upper and lower surfaces of the airfoil
    y_upper = airfoil_df[[f'y_upper_{i}' for i in range(n)]].values[0]
    y_lower = airfoil_df[[f'y_lower_{i}' for i in range(n)]].values[0]

    # Create interpolation functions for the upper and lower surfaces
    upper_interp_funcs[airfoil] = interp1d(x_values, y_upper, kind='cubic')
    lower_interp_funcs[airfoil] = interp1d(x_values, y_lower, kind='cubic')

In [17]:
# To test, return a few values to validate the interpolation functions for the first airfoil
airfoil_test = airfoils_names[1]
upper_sample = upper_interp_funcs[airfoil_test](x_values[:5])
lower_sample = lower_interp_funcs[airfoil_test](x_values[:5])

print(upper_sample, lower_sample)

[0.01865    0.02602718 0.03291268 0.03671403 0.04019054] [0.01865    0.01282987 0.00749698 0.00522047 0.00336181]


In [18]:
from scipy.misc import derivative

# Function to compute the derivative of the interpolated function
def compute_derivative(interp_function, x_values):
    return [derivative(interp_function, x_val, dx=1e-6) for x_val in x_values]

# Dictionaries to store the derivative values for all airfoils
upper_derivatives_all = {}
lower_derivatives_all = {}

# Loop through each airfoil name
for airfoil in airfoils_names:
    # Compute the derivatives while avoiding the boundary values
    upper_derivatives = compute_derivative(upper_interp_funcs[airfoil], x_values[1:-1])
    lower_derivatives = compute_derivative(lower_interp_funcs[airfoil], x_values[1:-1])
    
    # Add NaN to the beginning and end of the derivative lists to match the original size
    upper_derivatives_all[airfoil] = [np.nan] + upper_derivatives + [np.nan]
    lower_derivatives_all[airfoil] = [np.nan] + lower_derivatives + [np.nan]

# To test, return a few derivative values to validate for the first airfoil
airfoil_test = airfoils_names[0]
upper_derivatives_sample = upper_derivatives_all[airfoil_test][:5]
lower_derivatives_sample = lower_derivatives_all[airfoil_test][:5]

print(upper_derivatives_sample, lower_derivatives_sample)

[nan, 1.6163465331991285, 1.2095569291593196, 0.9257257503683669, 0.8679400729620645] [nan, -0.46147592699810963, -0.11754814226985419, 0.11506849556357224, 0.09407415609008485]


In [19]:
# Prepare a list to store the data
data = []

# Loop through each airfoil name
for airfoil in airfoils_names:
    # Create a dictionary for each airfoil
    airfoil_data = {'AIRFOIL NAME': airfoil}
    
    # Populate the dictionary with upper and lower derivatives
    for idx, value in enumerate(upper_derivatives_all[airfoil]):
        airfoil_data[f'upper_derivative_{idx}'] = value
    
    for idx, value in enumerate(lower_derivatives_all[airfoil]):
        airfoil_data[f'lower_derivative_{idx}'] = value
        
    # Append the dictionary to the data list
    data.append(airfoil_data)

# Convert the list of dictionaries into a DataFrame
df_derivatives = pd.DataFrame(data)

# Write the DataFrame to Excel
df_derivatives.to_excel('airfoil_derivatives2.xlsx', index=False, engine='openpyxl')